In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

In [2]:
import os
#path = "~/data/Data Files"
#test_path = "~/data/Random Sample of Data files"
test_df_path = 'random subset of data files/responses.csv'
test_cpoints_path = 'random subset of data files/checkpoints.csv'

In [3]:
def read_checkpoints(df_path):
    df = pd.read_csv(df_path)
    global students, students_unique, response, construct, chap_num, EOC
    students = df["student_id"].tolist()
    students_unique = list(set(df["student_id"]))
    response = df["response"].tolist()
    construct = df["construct"].tolist()
    chap_num = df["chapter_number"].tolist()
    EOC = df["EOC"].tolist()

def do_checkpoints(chapter):
    EOC_filtered = [EOC[i] for i in range(len(EOC)) if (not math.isnan(EOC[i])) and 
                    (chap_num[i]==chapter and construct[i]=="Expectancy")]
    if (len(EOC_filtered)==0):
        print(f"Chapter {chapter} Skipped: Has no corresponding pulse questions.")
        return []
    EOC_filtered.sort()
    EOC_checkpoints = EOC_filtered[::len(EOC_filtered)//6]
    EOC_checkpoints = EOC_checkpoints[:6]
    EOC_checkpoints.append(EOC_filtered[len(EOC_filtered)-1])
    return EOC_checkpoints

# returns actual score category of specific student
def group_student(value,cp):
    i=0
    while value>=cp[i]:
        i+=1
        if i==6:
            break
    return i

# Input: Chapter number
# Output: returns dictionary of student_id:[expected_score,actual_score]
# expected_score and actual_score are between [1,6]
def mk_groups(chapter):
    chapter = chapter+1
    correct_indexes = [i for i in range(len(students)) if chap_num[i]==chapter and construct[i]=="Expectancy"]
    checkpoints = do_checkpoints(chapter)
    if len(checkpoints)==0:
        return
    output_dict = {}
    for index in correct_indexes:
        stu = students[index]
        expected = response[index]
        if math.isnan(expected):
            continue
        actual = group_student(EOC[index],checkpoints)
        output_dict[stu] = [int(expected),actual]
    return output_dict

def mk_groups_all():
    chapters_all = list(set(chap_num))
    chapters_all.sort()
    chapters_all = chapters_all[:len(chapters_all)-1]
    dict_all = dict((c, mk_groups(c)) for c in chapters_all)
    return dict_all

In [4]:
read_checkpoints(test_cpoints_path) #input file path here
mydict = mk_groups(10) #chapter number
mystudent_id = 'a51af53e-6e4c-4651-832d-e7349e808a6d'
print(mydict[mystudent_id]) # for chapter 10, mystudent expected to achieve 3, but actual grade was 4

[3, 4]


In [5]:
mydict_all = mk_groups_all()

Chapter 14 Skipped: Has no corresponding pulse questions.
Chapter 15 Skipped: Has no corresponding pulse questions.
Chapter 16 Skipped: Has no corresponding pulse questions.


In [6]:
mydict_all[1]

{'988aaef5-55d8-4efd-a2aa-8a17e687965e': [4, 2],
 'bb3d0834-83be-4158-8d8e-6230178074a8': [4, 1],
 'c985759c-1ed2-4964-8c29-e17a0844e44a': [5, 3],
 '0556b794-cd80-4af6-8cfa-8055462c215a': [6, 5],
 '062f021e-8297-41e6-a8fe-54a382ab7404': [1, 6],
 '0d8dff90-f271-49ee-8e0e-cf7e83c7de61': [6, 6],
 '0f109cdd-f04e-4799-9a83-39ccebf8b171': [5, 4],
 '0f122889-830d-40c1-a6f8-f432cb327190': [6, 6],
 '32683ad9-2845-4977-9912-2c59bf0f6aa6': [5, 4],
 '3cc14c3e-8586-4017-80c1-152954ebbe2a': [6, 5],
 '41422302-20a7-417e-88ea-3b29631a418d': [4, 3],
 '4877fc0b-61be-404d-b1cf-bfa0e547feca': [4, 6],
 '4ca965ca-1794-4f5f-a14a-4a0ba41137d3': [2, 6],
 '53af77c6-6730-4c74-beb0-f717439e6e5a': [5, 3],
 '5f61c579-eaf0-4acd-ab54-b4143887a30e': [6, 6],
 '667f325c-12ee-4b85-807c-fe69fa3d92bb': [5, 2],
 '78ccab12-8733-4772-a2e0-17c7e68a69d6': [6, 5],
 '7a2cff82-cc76-44d0-bba7-0c8952982990': [5, 6],
 '7d467286-5c8a-40db-ace7-9f15e7f97d4d': [4, 1],
 '7d7a5092-b244-44a2-a776-3bb2c8d5a80f': [5, 4],
 '82a3e3eb-3f80-48b6

In [7]:

def eval_student(student_data):
    points = student_data[1] - student_data[0]
    if points > 0:
        return 1 # actual > expectancy
    if points < 0:
        return -1 # actual < expectancy
    return 0 # actual = expectancy


def eval_student_4(student_data):
    points = [student_data[0]-3.5, student_data[1]-3.5]
    if points[0]<0 and points[1]<0:
        return -2 # expected bad, did bad
    if points[0]<0 and points[1]>0:
        return -1 # expected bad, did well
    if points[0]>0 and points[1]<0:
        return 1 # expected well, did bad
    if points[0]>0 and points[1]>0:
        return 2 # expected well, did well

def eval_dict(student_dict):
    new_dict = {}
    for stu in student_dict:
        new_dict[stu] = eval_student(student_dict[stu])
    return new_dict

def eval_dict_4(student_dict):
    new_dict = {}
    for stu in student_dict:
        new_dict[stu] = eval_student_4(student_dict[stu])
    return new_dict

def categorization_to_id(student_dict):
    my_dict = eval_dict(student_dict)
    new_dict = {-1:[],0:[],1:[]}
    for stu in my_dict:
        new_dict[my_dict[stu]].append(stu)
    return new_dict
        
def categorization_to_id_4(student_dict):
    my_dict = eval_dict_4(student_dict)
    new_dict = {-2:[],-1:[],1:[],2:[]}
    for stu in my_dict:
        new_dict[my_dict[stu]].append(stu)
    return new_dict

In [8]:
new_dict = categorization_to_id(mydict)
new_dict

{-1: ['0f109cdd-f04e-4799-9a83-39ccebf8b171',
  '32683ad9-2845-4977-9912-2c59bf0f6aa6',
  '41422302-20a7-417e-88ea-3b29631a418d',
  '5f61c579-eaf0-4acd-ab54-b4143887a30e',
  '667f325c-12ee-4b85-807c-fe69fa3d92bb',
  '78ccab12-8733-4772-a2e0-17c7e68a69d6',
  '7d467286-5c8a-40db-ace7-9f15e7f97d4d',
  '7d7a5092-b244-44a2-a776-3bb2c8d5a80f',
  'a3ac1e2d-7dc6-4ff1-a8df-bb6ff0f52e67',
  'c7bb202e-6a69-4a84-9814-e55e6bd6d54b',
  'f52bcfe1-c230-4093-ae99-c46781171178',
  '5e037301-b06b-4b91-896b-fa411d055f7e',
  '7586f20a-8b0e-4467-8280-f68cbd64d1b0',
  '4d6f482c-40e1-43f1-a1df-955b2b65f10b',
  '691ba2d2-b3f9-4ded-b51c-231bdc0790da',
  'cce1092c-18e7-4698-8494-7e7a6ae5c695',
  'dd3499c0-5a33-4faf-b245-637a87138661',
  '0b20a3fe-7868-4115-ad82-eabccdd805c5',
  '1742e351-d617-48ed-a58f-817488831bc0',
  '19b2917a-72dc-4b25-ade1-eae2c2c7c036',
  '52e06937-6b52-45f4-8ad9-974a9efc0df0',
  '6d74553f-f6d1-4599-9e5d-a57b79ec695a',
  'c57b5e43-11ba-4ec7-b911-2085b009e948',
  'defb1dd0-c736-4a77-b0ac-0d9

In [9]:
new_dict = categorization_to_id_4(mydict)
new_dict

{-2: ['4ca965ca-1794-4f5f-a14a-4a0ba41137d3',
  '7d7a5092-b244-44a2-a776-3bb2c8d5a80f',
  '691ba2d2-b3f9-4ded-b51c-231bdc0790da',
  '2f429149-4815-40af-9308-3e26f0320822',
  '52e06937-6b52-45f4-8ad9-974a9efc0df0',
  'ba499004-ccbf-42ce-92e9-3c8c38f088d8',
  '9dd7096f-7041-4a6d-89d8-622012859d20',
  'e41950b0-6808-49bb-8f0c-70501cea596f',
  '84b602c6-252b-44e7-964d-1cbe06e5ea85',
  '44c4b514-7b57-42eb-a4f7-defa1fb249aa',
  'f77e9e18-7090-4e4a-af77-114505b29e97',
  'ab915c05-cd34-40bf-b4fd-321ac00304ec'],
 -1: ['062f021e-8297-41e6-a8fe-54a382ab7404',
  '882f55fd-273b-4df7-bd91-4eb3a246d0ab',
  '97a884d8-9c0b-446d-922a-812553ccb4fd',
  '8300403a-4e77-4300-9a20-ced3926b687c',
  'a51af53e-6e4c-4651-832d-e7349e808a6d'],
 1: ['32683ad9-2845-4977-9912-2c59bf0f6aa6',
  '5f61c579-eaf0-4acd-ab54-b4143887a30e',
  '667f325c-12ee-4b85-807c-fe69fa3d92bb',
  '78ccab12-8733-4772-a2e0-17c7e68a69d6',
  '7d467286-5c8a-40db-ace7-9f15e7f97d4d',
  'a3ac1e2d-7dc6-4ff1-a8df-bb6ff0f52e67',
  '5e037301-b06b-4b91